In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. Allostatic Load/P1012.3. Analyses/P1012.3.1. Mediator Score Calculation/Output')
cur_date = "061424"

library(readxl)
library(openxlsx)
library(tidyverse)
library(imputeLCMD)

# reading in file
cytokine_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 2)) 
bp_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 4)) 
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

In [2]:
# creating 1 df
full_df = inner_join(subject_info_df, cytokine_df) #%>%
    # filtering for subjects who have blood pressure measurements
    #filter(Subject_ID %in% unique(bp_df$Subject_ID))
head(full_df)

Joining with `by = join_by(Subject_ID)`


,Study,Original_Subject_Number,Subject_ID,Subject_Number,Smoking_Status,Sex,Age,Race,Category,Variable,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,FM2,97,CS_M_18_B_1,1,CS,M,18,B,AL Biomarker,Cortisol,140.304
2,FM2,97,CS_M_18_B_1,1,CS,M,18,B,AL Biomarker,Noradrenaline,2638.625
3,FM2,97,CS_M_18_B_1,1,CS,M,18,B,AL Biomarker,Hba1c,3000.829
4,FM2,97,CS_M_18_B_1,1,CS,M,18,B,AL Biomarker,Fibrinogen,944805.237
5,FM2,97,CS_M_18_B_1,1,CS,M,18,B,AL Biomarker,CRP,55598.454
6,FM2,97,CS_M_18_B_1,1,CS,M,18,B,Cytokine,IP10,0.099


Starting by figuring out the amount of missing data first.

In [3]:
# only keeping variables with at least 25% of data 
variable_presence_df = full_df %>% 
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(Variable) %>%
    summarize(Variable_Presence_Percent = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence_Percent)

head(variable_presence_df)

Variable,Variable_Presence_Percent
<chr>,<dbl>
Epinephrine,82.53968
CRP,100.00000
Cortisol,100.00000
Fibrinogen,100.00000
HDL,100.00000
Hba1c,100.00000


Epinephrine is the only variable with missing data with ~83% of data present. The data was missing due to it being below the limit of detection so we'll use QRILC imputation.

# QRILC Imputation

In [4]:
# imputing soluble mediator data using QRILC
QRILC_imputation = function(dataset){
    wider_dataset = dataset %>%
        # removing this column for now since it's giving me issues
        select(-Category) %>%
        pivot_wider(names_from = Variable, values_from = Value)
    
    index_of_last_variable = length(colnames(wider_dataset))
    
    # normalizing data since that what the QRILC function wants
    # had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,9:all_of(index_of_last_variable)] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
                   
    imputed_dataset = data.frame(cbind(unique(dataset[,1:8]), QRILC_df)) %>%
        pivot_longer(cols = 9:all_of(index_of_last_variable), names_to = "Variable", values_to = "Value")
    
    return(imputed_dataset)
}

# calling fn
imputed_df = QRILC_imputation(full_df)

head(imputed_df)

Warning message:
“Using `all_of()` outside of a selecting function was deprecated in tidyselect
1.2.0.
ℹ See details at
  <https://tidyselect.r-lib.org/reference/faq-selection-context.html>”


Study,Original_Subject_Number,Subject_ID,Subject_Number,Smoking_Status,Sex,Age,Race,Variable,Value
<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
FM2,97,CS_M_18_B_1,1,CS,M,18,B,Cortisol,140.304
FM2,97,CS_M_18_B_1,1,CS,M,18,B,Noradrenaline,2638.625
FM2,97,CS_M_18_B_1,1,CS,M,18,B,Hba1c,3000.829
FM2,97,CS_M_18_B_1,1,CS,M,18,B,Fibrinogen,944805.237
FM2,97,CS_M_18_B_1,1,CS,M,18,B,CRP,55598.454
FM2,97,CS_M_18_B_1,1,CS,M,18,B,IP10,0.099


In [5]:
mediator_score = function(df){
    # """
    # Creating a scoring function for each mediator.
    # :param (input): initial df (df), smoking status, and race
    # :output: df containing the variable (biomarker) name, subject ID, and score
    # """
    
    # creating an empty df to store values
    score_df = data.frame()
    
    # getting all variable names for loop to iterate through
    mediators = unique(df$Variable)
    
    for (i in 1:length(mediators)){

        # filtering df for each mediator
        filtered_df = df %>%
            filter(Variable == mediators[i]) %>%
            # added this since epinephrine had missing data
            drop_na()
        
        # now iterating through each value of the filtered_df
        for (j in 1:length(filtered_df$Value)){

            # score = (mediator value - mediator min)/ (mediator max - mediator min)
            mediator_score_formula = (filtered_df$Value[j] - min(filtered_df$Value))/(max(filtered_df$Value) - min(filtered_df$Value))

            # storing mediator, subject id, and score
            values_vector = cbind(mediators[i], filtered_df$Subject_ID[j], mediator_score_formula)
            score_df = rbind(score_df, values_vector)
        }
    }
    
    # renaming columns
    colnames(score_df) = c("Variable", "Subject_ID", "Mediator_Score")
    
    # for some reason the Mediator_Score is a character type, so changing to a numeric
    score_df$Mediator_Score = as.numeric(score_df$Mediator_Score)
    
    return(score_df)
}

In [6]:
# calling function
mediator_score_df = mediator_score(imputed_df)

head(mediator_score_df)

,Variable,Subject_ID,Mediator_Score
,<chr>,<chr>,<dbl>
1,Cortisol,CS_M_18_B_1,0.4683470
2,Cortisol,CS_F_20_W_2,0.8342112
3,Cortisol,CS_F_20_W_3,0.1603803
4,Cortisol,CS_F_21_W_4,0.4583014
5,Cortisol,CS_M_21_W_5,0.3867217
6,Cortisol,CS_F_22_W_6,0.1712352


In [7]:
# exporting results
write.xlsx(mediator_score_df, paste0(Output,"/", "Mediator_Scores_", cur_date, ".xlsx"), rowNames = FALSE)